##User interface: R Shiny Application 

In [6]:
import TIdatabase as ti
import pandas as pd

%load_ext rpy2.ipython

In [8]:
%%R 
a=c(1,2,3)
print(a)

[1] 1 2 3


In [ ]:
%%R
library(ggvis)
library(dplyr)

# For dropdown menu
actionLink <- function(inputId, ...) {
  tags$a(href='javascript:void',
         id=inputId,
         class='action-button',
         ...)
}

shinyUI(fluidPage(
  titlePanel("What are my chances?"),
  fluidRow(
    column(3,
           wellPanel(
             h4("Application information"),
             sliderInput("sat", "SAT score (combined)",
                         0, 2400, 1500, step = 10),
             sliderInput("gpa", "GPA", 0, 5, step = .1),
             sliderInput("rank", "Class Rank (percentile)",
                         0, 100, 0, step = 5),
             selectInput("apnum", "Number of AP exams taken",
                         c("0", "1", "2", "3", "4", "5",
                           "6", "7", "8", "9", "10", "11",
                           "12", "Music", "Musical", "Mystery", "Romance", "Sci-Fi",
                           "Short", "Sport", "Thriller", "War", "Western")),
             selectInput("genre", "Genre (a movie can have multiple genres)",
                         c("All", "Action", "Adventure", "Animation", "Biography", "Comedy",
                           "Crime", "Documentary", "Drama", "Family", "Fantasy", "History",
                           "Horror", "Music", "Musical", "Mystery", "Romance", "Sci-Fi",
                           "Short", "Sport", "Thriller", "War", "Western")
             ),
             textInput("college", "What college are you applying to?"),
             textInput("cast", "Cast names contains (e.g. Tom Hanks)")
           ),
           #wellPanel(
             #selectInput("xvar", "X-axis variable", axis_vars, selected = "Meter"),
             #selectInput("yvar", "Y-axis variable", axis_vars, selected = "Reviews"),
             #tags$small(paste0(
               "Note: This predictor tool is based on data from only a sample of students",
               " and does not provde a guarantee of admission to any institution."
               "
             ))
           )
    ),
    column(9,
           ggvisOutput("plot1"),
           wellPanel(
             span("Number of movies selected:",
                  textOutput("n_movies")
             )
           )
    )
  )
))
